In [1]:
!pip install nltk

In [23]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
Customer_dataset = pd.read_csv('/content/Customer reviews.csv')
Customer_dataset.head(10)

,Title,Contents,Date
0,Quality customer service,My order was left outside and the school kids ...,"Dec 14, 2019"
1,Helpful and friendly staff.,Helpful and friendly staff.,"Dec 14, 2019"
2,Replied quickly and made sure their…,Replied quickly and made sure their customers ...,"Dec 14, 2019"
3,Helpful customer service officer but problemat...,Rebecca was patient and helpful with my querie...,"Dec 14, 2019"
4,"Perfect, love ASOS's service","Fast, kind, clear answer! Perfect that I love ...","Dec 14, 2019"
5,It took over 20 minutes to be connected…,It took over 20 minutes to be connected with t...,"Dec 14, 2019"
6,Terrible firm,Terrible firm. I had such a bad time trying to...,"Dec 14, 2019"
7,very helpful,"very helpful, clear and prompt","Dec 14, 2019"
8,Delivery but no watch inside!,Ordered a Vivienne Westwood which was delivere...,"Dec 14, 2019"
9,Terrible,Terrible Paid for next day delivery and I stil...,"Dec 14, 2019"


In [5]:
Customer_dataset['Title'].value_counts()

Title
Great customer service                                 20
Very helpful                                           17
Great service                                          11
Helpful                                                 9
Great customer service!                                 7
                                                       ..
One of my favorite stores                               1
There was a problem with delivery but Misty helped!     1
Liam was amazing!!                                      1
NARINA WAS AN AMAZING HELP TODAY WITH…                  1
Great after purchase service                            1
Name: count, Length: 1833, dtype: int64

In [6]:
Customer_dataset.shape

(1999, 3)

In [7]:
Customer_dataset.describe(include='all')

,Title,Contents,Date
count,1999,1999,1999
unique,1833,1961,30
top,Great customer service,Very helpful,"Dec 2, 2019"
freq,20,6,197


In [8]:
Customer_dataset.isnull().sum()

Title       0
Contents    0
Date        0
dtype: int64

In [9]:
new_data1 = Customer_dataset.copy()

In [10]:
data = pd.DataFrame({
    'title': new_data1['Title'],
    'contents': new_data1['Contents'],
    'sentiment': ["positive", "negative", "positive", "negative"] * (len(new_data1) // 4) + ["positive", "negative", "positive"] # Adjust the repetition and add remaining elements
})

In [11]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [12]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

data['tokenized_title'] = data['title'].apply(preprocess_text)
data['tokenized_content'] = data['contents'].apply(preprocess_text)
data['tokenized_text'] = data.apply(lambda row: row['tokenized_title'] + row['tokenized_content'], axis=1)

In [13]:
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform([' '.join(tokens) for tokens in data['tokenized_text']])

In [14]:
label_mapping = {'positive': 1, 'negative': 0}
y = data['sentiment'].map(label_mapping)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
model = SVC(kernel='linear', C=1.0)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy_score = accuracy_score(y_test, y_pred) * 100
print("Accuracy:", accuracy_score)

Accuracy: 51.74999999999999


In [24]:
# Accuracy Score
accuracy_score = accuracy_score(y_test, y_pred) * 100
print("Accuracy:", accuracy_score)

# precision Score
precision = precision_score(y_test, y_pred)*100
print("Precision:", precision)

# F1 Score
F1 = f1_score(y_test, y_pred)*100
print("F1_Score:", F1)

#Recall
recall = recall_score(y_test, y_pred)*100
print("Reacll_Score:", recall)

# Confusion Matrix
confusion_matrix = pd.crosstab(y_test, y_pred)
print("Confusion Matrix:\n", confusion_matrix)

Accuracy: 51.74999999999999
Precision: 52.0
F1_Score: 54.800936768149874
Reacll_Score: 57.920792079207914
Confusion Matrix:
 col_0       0    1
sentiment         
0          90  108
1          85  117


In [25]:
classf_report = classification_report(y_test, y_pred)
print("Classification Report:\n", classf_report)

Classification Report:
               precision    recall  f1-score   support

           0       0.51      0.45      0.48       198
           1       0.52      0.58      0.55       202

    accuracy                           0.52       400
   macro avg       0.52      0.52      0.52       400
weighted avg       0.52      0.52      0.52       400

